In [1]:
import pandas as pd
from uszipcode import SearchEngine
import numpy as np

In [2]:
houston_zip_codes = pd.read_csv('./Data/houston_cleaned_zipcodes.csv')

In [3]:
houston_zip_codes.head()

,Zip_Code,Name
0,77002,"Downtown, Midtown"
1,77003,East Downtown
2,77004,"South Central, Greater Third Ward"
3,77005,West University Place
4,77006,"Montrose, Hyde Park"


In [4]:
dfw_zipcodes = pd.read_csv('./Data/dfw_cleaned_zipcodes.csv')

In [5]:
dfw_zipcodes.head()

,Zip_Code,Name
0,75001,Addison
1,75002,West Allen
2,75006,Central Carrolton
3,75007,North Carrollton
4,75009,Celina


In [6]:
def get_demographics(zipcode_df):
    demos = []
    for zipcode in zipcode_df['Zip_Code']:
        full_zipcode_db = SearchEngine(simple_zipcode = False)
        zipcode_demographics = full_zipcode_db.by_zipcode(zipcode)
        zipcode_demographics = zipcode_demographics.to_dict()
        keys = ['lat', 'lng', 'population', 'median_household_income', 'median_home_value']
        demo_list = [zipcode_demographics.get(key) for key in keys]
        education_column_names = ["Less Than High School", "High School Graduate", "Associate's Degree", "Bachelor's Degree", "Master's Degree", "Professional Degree", "Doctorate Degree"]
        education_values = [zipcode_demographics['educational_attainment_for_population_25_and_over'][0]['values'][i]['y'] for i in range(len(education_column_names))]
        demo_list = demo_list + education_values
        keys = keys + education_column_names
        demos.append(demo_list)
    df = pd.DataFrame(demos, columns = keys)
    df = pd.concat([zipcode_df, df], axis = 1)
    return df

In [7]:
dallas_df = get_demographics(dfw_zipcodes)

In [8]:
dallas_df.head()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,75001,Addison,32.96,-96.84,12414,57633.0,258200.0,919,3655,381,3318,1439,232,34
1,75002,West Allen,33.08,-96.61,63140,91632.0,186100.0,2136,16961,3465,12670,4453,625,434
2,75006,Central Carrolton,32.95,-96.89,46364,56985.0,146800.0,7814,11857,2041,6459,2326,315,80
3,75007,North Carrollton,33.00,-96.90,51624,78934.0,169400.0,3485,14680,2810,9935,2614,511,416
4,75009,Celina,33.30,-96.80,8785,87581.0,190500.0,420,2991,608,1230,393,35,18


In [9]:
def education_percentage(df):
    education_columns = df[['Less Than High School', 'High School Graduate', "Associate's Degree", "Bachelor's Degree", "Master's Degree", 'Professional Degree', 'Doctorate Degree']]
    education_percentages = education_columns.div(education_columns.sum(axis = 1), axis = 0)
    df.update(education_percentages)
    return df

In [10]:
dallas_df = education_percentage(dallas_df)

In [11]:
dallas_df.head()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,75001,Addison,32.96,-96.84,12414,57633.0,258200.0,0.092103,0.366306,0.038184,0.332532,0.144217,0.023251,0.003407
1,75002,West Allen,33.08,-96.61,63140,91632.0,186100.0,0.052425,0.416282,0.085043,0.310966,0.109292,0.015340,0.010652
2,75006,Central Carrolton,32.95,-96.89,46364,56985.0,146800.0,0.252946,0.383821,0.066069,0.209083,0.075295,0.010197,0.002590
3,75007,North Carrollton,33.00,-96.90,51624,78934.0,169400.0,0.101158,0.426112,0.081565,0.288381,0.075876,0.014833,0.012075
4,75009,Celina,33.30,-96.80,8785,87581.0,190500.0,0.073749,0.525198,0.106760,0.215979,0.069008,0.006146,0.003161


In [12]:
dallas_df.to_csv('./Data/dfw_demographics.csv', index = False)

In [13]:
houston_df = get_demographics(houston_zip_codes)

In [14]:
houston_df.head()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,77002,"Downtown, Midtown",29.75,-95.37,16793,64093.0,208600.0,2413,3778,446,1389,651,197,206
1,77003,East Downtown,29.75,-95.35,10508,36412.0,162600.0,2191,2544,266,1141,421,202,122
2,77004,"South Central, Greater Third Ward",29.72,-95.38,32692,41983.0,191700.0,2998,6567,769,4387,2607,1146,540
3,77005,West University Place,29.72,-95.42,25528,159732.0,754000.0,262,1997,661,5473,3716,3008,1224
4,77006,"Montrose, Hyde Park",29.74,-95.39,19664,67458.0,327300.0,557,4375,512,5607,2845,1674,969


In [15]:
houston_df = education_percentage(houston_df)

In [16]:
houston_df.head()

,Zip_Code,Name,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,77002,"Downtown, Midtown",29.75,-95.37,16793,64093.0,208600.0,0.265749,0.416079,0.049119,0.152974,0.071696,0.021696,0.022687
1,77003,East Downtown,29.75,-95.35,10508,36412.0,162600.0,0.318136,0.369392,0.038623,0.165674,0.061130,0.029331,0.017715
2,77004,"South Central, Greater Third Ward",29.72,-95.38,32692,41983.0,191700.0,0.157673,0.345377,0.040444,0.230725,0.137109,0.060271,0.028400
3,77005,West University Place,29.72,-95.42,25528,159732.0,754000.0,0.016033,0.122208,0.040450,0.334924,0.227403,0.184077,0.074904
4,77006,"Montrose, Hyde Park",29.74,-95.39,19664,67458.0,327300.0,0.033678,0.264526,0.030957,0.339017,0.172018,0.101215,0.058589


In [17]:
houston_df.to_csv('./Data/houston_demographics.csv', index = False)